In [7]:
import os
import pandas as pd
import duckdb

projects = []

log_dir = "logs/"
for filename in os.listdir(log_dir):
    if filename.endswith("_all_files.csv"):
        task_id = filename.replace("_all_files.csv", "")
        projects.append(task_id)

df_report = pd.DataFrame(projects, columns=['project'])

In [8]:
df_report = duckdb.query(f"""
    select r.project, rr.GitHub from df_report r
        left join repositories.csv rr on r.project = rr.Project
""").to_df()

In [9]:
df_report = duckdb.query(f"""
    select r.*, ra.R1, ra.R5, ra.R10 from df_report r
        left join reliability_projects.csv ra on r.project = ra.project
""").to_df()

In [10]:
df_summary = pd.DataFrame()

log_dir = "analysis/"
for filename in os.listdir(log_dir):
    if filename.endswith("_logfile-summary.csv"):
        task_id = filename.replace("_logfile-summary.csv", "")
        try:
            df_summary_line = pd.read_csv(f"analysis/{filename}")
            df_summary_line = df_summary_line.set_index('statistic').T
            df_summary_line['project'] = task_id
            df_summary = pd.concat([df_summary, df_summary_line], ignore_index=True)
        except: 
            print(f"Erro ao ler o arquivo analysis/{filename}")
            pass

df_report = duckdb.query(f"""
    select r.*, s."number-of-commits", s."number-of-entities", s."number-of-entities-changed", s."number-of-authors" 
        from df_report r
        left join df_summary s on r.project = s.project
""").to_df()

Erro ao ler o arquivo analysis/commons-io_logfile-summary.csv
Erro ao ler o arquivo analysis/jspwiki_logfile-summary.csv
Erro ao ler o arquivo analysis/graal_logfile-summary.csv
Erro ao ler o arquivo analysis/zookeeper_logfile-summary.csv
Erro ao ler o arquivo analysis/poi_logfile-summary.csv
Erro ao ler o arquivo analysis/freeplane_logfile-summary.csv
Erro ao ler o arquivo analysis/quartz_logfile-summary.csv


In [11]:
df_summary = pd.DataFrame()


log_dir = "analysis/"
for filename in os.listdir(log_dir):
    if filename.endswith("_logfile-age.csv"):
        task_id = filename.replace("_logfile-age.csv", "")
        #print(filename)
        try:
            df_task = pd.DataFrame()
            
            df_task = duckdb.query(f"""
                select max("age-months") as "max-age-months", avg("age-months") as "avg-age-months"
                    from "{log_dir}{filename}" r
            """).to_df()

            df_task['project'] = task_id
            df_summary = pd.concat([df_summary, df_task], ignore_index=True)
        except: 
            print(f"Erro ao ler o arquivo analysis/{filename}")
            pass

df_report = duckdb.query(f"""
    select r.*, t."max-age-months", t."avg-age-months" 
        from df_report r
        left join df_summary t on r.project = t.project
""").to_df()

Erro ao ler o arquivo analysis/zookeeper_logfile-age.csv
Erro ao ler o arquivo analysis/quartz_logfile-age.csv
Erro ao ler o arquivo analysis/poi_logfile-age.csv
Erro ao ler o arquivo analysis/commons-io_logfile-age.csv
Erro ao ler o arquivo analysis/graal_logfile-age.csv
Erro ao ler o arquivo analysis/jspwiki_logfile-age.csv
Erro ao ler o arquivo analysis/freeplane_logfile-age.csv


In [12]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None,'display.max_colwidth', -1):
    display(df_report)

,project,GitHub,R1,R5,R10,number-of-commits,number-of-entities,number-of-entities-changed,number-of-authors,max-age-months,avg-age-months
0,cglib,https://github.com/cglib/cglib.git,0.9807,0.9073,0.8232,824.0,986.0,3887.0,29.0,268.0,202.482759
1,giraph,https://github.com/apache/giraph.git,0.9310,0.6996,0.4894,1128.0,4313.0,11888.0,49.0,171.0,132.129144
2,java-design-patterns,https://github.com/iluwatar/java-design-patterns.git,0.9327,0.7057,0.4981,3713.0,6265.0,34138.0,505.0,126.0,51.320511
3,opengrok,https://github.com/oracle/opengrok.git,0.8000,0.3278,0.1074,6975.0,7679.0,32957.0,152.0,219.0,78.275687
4,takes,https://github.com/yegor256/takes.git,0.8812,0.5312,0.2822,2764.0,1000.0,11920.0,116.0,119.0,48.892000
5,Firestorm,https://github.com/Tencent/Firestorm.git,0.9311,0.6997,0.4895,141.0,461.0,1642.0,14.0,39.0,34.704989
6,checkstyle,https://github.com/checkstyle/checkstyle.git,0.7614,0.2559,0.0655,14573.0,16618.0,94226.0,540.0,284.0,64.393970
7,glide,https://github.com/bumptech/glide.git,0.9209,0.6622,0.4385,2766.0,4026.0,19702.0,191.0,146.0,97.445107
8,javaparser,https://github.com/javaparser/javaparser.git,0.8193,0.3691,0.1362,6568.0,10044.0,48352.0,268.0,159.0,83.670649
9,opennlp,https://github.com/apache/opennlp.git,0.9493,0.7708,0.5941,2233.0,5217.0,18169.0,80.0,170.0,123.581369
